In [1]:
import Pkg; Pkg.activate("../"); # Activate the project 

  Activating project at `~/Documents/Research/MonitoringMetrology/QuTaM.jl`


In [2]:
# Import all the necessary libraries
libraries = ["QuTaM", "LinearAlgebra"]

function ensure_imports(packages::Vector{String})
    for pkg in packages
        try
            @eval using $(Symbol(pkg))
        catch e
            if e isa ArgumentError && occursin(pkg, e.msg)
                println("Installing $pkg...")
                Pkg.add(pkg)
                @eval using $(Symbol(pkg))
            else
                rethrow(e)
            end
        end
    end
end

ensure_imports(libraries)

Precompiling QuTaM
  ✓ QuTaM
  1 dependency successfully precompiled in 6 seconds. 294 already precompiled.


In [3]:
H_parametrized = (delta::Float64, gamma::Float64) -> (0.5*delta*QuTaM.sigma_z)::Matrix{ComplexF64}
L_parametrized = (delta::Float64, gamma::Float64) -> (sqrt(gamma)*QuTaM.sigma_m)::Matrix{ComplexF64}
# Function that constructs returns a parametrization of the effective hamiltonian from 
# the parametrization of the hamiltonian and the jump operators
function GetHeffParametrized(H_par::Function, Ls_par)::Function
    return (input...) -> begin
        LLs_par = [ adjoint(L_par(input...))*L_par(input...) for L_par in Ls_par]
        return H_par(input...) - 0.5im*sum(LLs_par)  
    end 
end 

GetHeffParametrized (generic function with 1 method)

In [4]:
function DerivativeExpHeff(Heff_par::Function, tau::Float64, theta::Vector{Float64}, dtheta::Vector{Float64})
    f1 =  exp(-1im*tau*Heff_par((theta + 2*dtheta)...))
    f2 =  exp(-1im*tau*Heff_par((theta + 1*dtheta)...))
    f3 =  exp(-1im*tau*Heff_par((theta - 1*dtheta)...))
    f4 =  exp(-1im*tau*Heff_par((theta - 2*dtheta)...))
    return (-f1 + 8*f2 - 8*f3 + f4 )/(12*norm(dtheta))
end 

function DerivativesAtJumps(sys::System, Heff_par::Function, Ls_par, traj::Trajectory, psi0::Vector{ComplexF64}, theta::Vector{Float64},
                            dtheta::Vector{Float64})
    # 1. Get the derivatives of L
    nchannels = size(Ls_par)[1]
    dLs = zeros(ComplexF64, sys.NLEVELS, sys.NLEVELS, nchannels)
    
    for k in 1:nchannels
        f1 = Ls_par[k]((theta + 2*dtheta)...) 
        f2 = Ls_par[k]((theta+dtheta)...) 
        f3 = Ls_par[k]((theta-dtheta)...)
        f4 = Ls_par[k]((theta-2*dtheta)...)
        dLs[:, :, k] = (-f1 + 8*f2 - 8*f3 + f4 )/(12*norm(dtheta))
    end 
    # 2.1 Setup 
    njumps = size(traj)[1]
    tmp1 = copy(psi0)
    tmp2 = zeros(ComplexF64, sys.NLEVELS) 
    dpsis = zeros(ComplexF64, sys.NLEVELS, njumps)
    # 2.2 Set up the first jump
    click = traj[1] 
    label = click.label
    tau = click.time
    dpsis[:, 1] .= dLs[:, :, label] * exp(-1im*tau*sys.Heff) * tmp1 +
                      sys.Ls[label] * DerivativeExpHeff(Heff_par, tau, theta, dtheta) * tmp1 # Derivative
    # In case there are no more jumps, return
    if njumps  == 1 
        return dpsis
    end  
    # 3. Go over the rest of the jumps 
    psitildes = states_at_jumps(traj, sys, psi0; normalize=false)
    for k in 2:njumps 
      click = traj[k] 
      label = click.label
      tau = click.time
      # Calculate the derivative
      dpsis[:, k] .= (dLs[:, :, label]) * exp(-1im*tau*sys.Heff) * psitildes[k-1, :] +
                     sys.Ls[label] * DerivativeExpHeff(Heff_par, tau, theta, dtheta) * psitildes[k-1, :] # Derivative
                     sys.Ls[label] * exp(-1im*tau*sys.Heff) * dpsis[:, k -1]
   end 
   return dpsis 
    
end 

DerivativesAtJumps (generic function with 1 method)

In [5]:
Heff_parametrized = GetHeffParametrized(H_parametrized, [L_parametrized])
sys = QuTaM.rd_sys 
traj = [DetectionClick(0.1, 1)]
psi0 = [0.0 + 0im, 1.0+ 0im]
DerivativesAtJumps(sys, Heff_parametrized, [L_parametrized], traj, psi0, 
                    [QuTaM.rd_deltaomega, QuTaM.rd_gamma], [0.0, QuTaM.rd_gamma/100])

2×1 Matrix{ComplexF64}:
 0.17355820485444767 - 0.01129719801926627im
                 0.0 + 0.0im

In [6]:
0.5/sqrt(QuTaM.rd_gamma) * exp(-1im*traj[1].time*(0.5*QuTaM.rd_deltaomega - 0.5im*QuTaM.rd_gamma)) +
sqrt(QuTaM.rd_gamma)*(-0.5*traj[1].time)*exp(-1im*traj[1].time*(0.5*QuTaM.rd_deltaomega - 0.5im*QuTaM.rd_gamma))

0.17355820539697653 - 0.011297198054579793im